## INTENTO RECONOCIMIENTO MATRICULA VIDEO

In [2]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import pytesseract
from pytesseract import Output
import re
import numpy as np
import time


# Método que coloca dos imágenes en una misma figura
def plot_images(img1, img2, title1="", title2=""):
    fig = plt.figure(figsize=[15, 15])
    ax1 = fig.add_subplot(121)
    ax1.imshow(img1, cmap="gray")
    ax1.set(xticks=[], yticks=[], title=title1)
    ax2 = fig.add_subplot(122)
    ax2.imshow(img2, cmap="gray")
    ax2.set(xticks=[], yticks=[], title=title2)
    plt.show()

def plate_validation(plate):
    # Definir el patrón de la matrícula "0000 AAA"
    plate_pattern = re.compile(r'^\d{4}\s[A-Z]{3}$')

    if plate_pattern.match(plate):
        # La matrícula cumple con el formato
        return plate
    else:
        # La matrícula no cumple con el formato, intentar corregirla
        plate_refactored = re.sub(r'^\s*.*?(\d{4}\s[A-Z]{3}).*$', r'\1', plate)
        return plate_refactored

# Cargamos el modelo de YOLO
model_car = YOLO('yolov8n.pt')

# Guardamos la ruta absoluta de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

def process_frame(frame):
    # Convertir la imagen a RGB ya que OpenCV carga las imágenes en formato BGR
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detectar coches
    results_cars = model_car(frame_rgb, verbose=False)

    # Para cada detección de coche
    for r_car in results_cars:
        boxes_car = r_car.boxes

        for box_car in boxes_car:
            # Si la clase es un coche
            if int(box_car.cls[0]) == 2:
                # Coordenadas del cuadro del coche
                x1_car, y1_car, x2_car, y2_car = box_car.xyxy[0]

                # Definir la región inferior para buscar matrículas, deben estar dentro del cuadro del coche y en la parte central

                # Coordenadas de la región inferior
                x1_bottom, y1_bottom, x2_bottom, y2_bottom = int(x1_car), int(y2_car), int(x2_car), int(y2_car + (y2_car - y1_car))

                # Extraer la región inferior
                roi_bottom = frame[y1_bottom:y2_bottom, x1_bottom:x2_bottom]
                
                # Ajustar los valores según tus necesidades
                min_area = 100
                max_area = 5000
                
                # Aplicar umbralización adaptativa
                thresholded = cv2.adaptiveThreshold(cv2.cvtColor(roi_bottom, cv2.COLOR_BGR2GRAY), 255,
                                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

                # Encontrar contornos en la imagen umbralizada
                contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Filtrar contornos por área
                contours = [contour for contour in contours if cv2.contourArea(contour) > min_area and cv2.contourArea(contour) < max_area]

                # Buscar el contorno con el área máxima (presumiblemente la matrícula)
                max_contour = max(contours, key=cv2.contourArea) if contours else None

                # Si hay un contorno válido, obtener el rectángulo delimitador
                if max_contour is not None:
                    x, y, w, h = cv2.boundingRect(max_contour)

                    # Ajustar las coordenadas al tamaño original de la imagen
                    x, y, w, h = int(x + x1_car), int(y + (y1_car + y2_car) / 2), int(w), int(h)

                    # Extraer la región de la matrícula
                    plate = frame[y:y + h, x :x + w]

                    # Dibuja el contenedor y clase
                    cv2.rectangle(frame, (int(x1_car), int(y1_car)), (int(x2_car), int(y2_car)), (0, 0, 255), 4)
                    cv2.putText(frame, "Car", [int(x1_car), int(y1_car)], cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 5)

                    # Si se ha detectado una matrícula, se muestra en RGB
                    if plate is not None and plate.size > 0:
                        gray_plate = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
                        plate_preprocessed = cv2.equalizeHist(gray_plate)  # Aplicar el preprocesamiento aquí
                        plate_preprocessed_rgb = cv2.cvtColor(plate_preprocessed, cv2.COLOR_GRAY2RGB)
                        try:
                            text = pytesseract.image_to_string(plate_preprocessed_rgb, config='--psm 8', output_type=Output.STRING)
                            text = plate_validation(text)
                            print("\n[+] MATRÍCULA DETECTADA:", text)

                            # Mostrar el texto en la ventana del video
                            cv2.putText(frame, f"Plate: {text}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                            # Dibujar el rectángulo alrededor de la matrícula
                            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                        except Exception as e:
                            print("Error al procesar la matrícula:", e)

    # Mostrar el frame procesado
    cv2.imshow('Plate Detection', frame)
    key = cv2.waitKey(1)  # Esperar 1 milisegundo para que se actualice la ventana
    delay = 100

    time.sleep(delay / 1000.0)

    # Comprobar si se presionó la tecla 'q' para salir
    if key & 0xFF == ord('q'):
        return None

    return frame

def plate_detector_video(video_path):
    # Inicializar el objeto de captura de video
    cap = cv2.VideoCapture(video_path)

    # Obtener el ancho y alto del video
    width = int(cap.get(2))
    height = int(cap.get(3))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Cambiado de 'XVID' a 'mp4v' para formato MP4
    out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (width, height))

    # Leer frames del video
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Procesar el frame
        processed_frame = process_frame(frame)

        # Verificar si se cerró la ventana
        if processed_frame is None:
            break

        # Escribir el frame procesado en el archivo de salida
        out.write(processed_frame)

    # Liberar los objetos de captura y escritura
    cap.release()
    out.release()
    cv2.destroyAllWindows()  # Cerrar la ventana después de procesar todos los frames

# Procesar el video
plate_detector_video("video/video.mp4")
